In [50]:
from camp_networks import create_moria
from scipy.stats import poisson
from stats_utils import sample_population
from seirsplus.models import SymptomaticSEIRSNetworkModel
from network_utils import run_simulation
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [2]:
graph, node_groups=create_moria()

100%|██████████| 18700/18700 [00:04<00:00, 3858.89it/s]


In [9]:
n_pop = 18700

# Grid info for isoboxes
dims_isoboxes = (29, 28)  # 812

# Grid info for tents
dims_block1 = (20, 67)  # 1340
dims_block2 = (53, 15)  # 795
dims_block3 = (19, 28)  # 532

# Isoboxes
pop_isoboxes = 8100
pop_per_isobox = 10
n_isoboxes = dims_isoboxes[0]*dims_isoboxes[1]

# Tents
n_tents = 2650
pop_tents = 10600
pop_per_tent = 4

# Others 
n_bathrooms = 144
n_ethnic_groups = 8

# We define neighboring structures within a range of 2 in the structure grid
proximity = 2

# Define the maximum population per structures (tents and isoboxes) drawn from a poisson distribution
max_pop_per_struct = list(poisson.rvs(mu=pop_per_isobox, size=n_isoboxes))
max_pop_per_struct = max_pop_per_struct + list(poisson.rvs(mu=pop_per_tent, size=dims_block1[0]*dims_block1[1]))
max_pop_per_struct = max_pop_per_struct + list(poisson.rvs(mu=pop_per_tent, size=dims_block2[0]*dims_block2[1]))
max_pop_per_struct = max_pop_per_struct + list(poisson.rvs(mu=pop_per_tent, size=dims_block3[0]*dims_block3[1]))

n_structs = len(max_pop_per_struct)

# Sample the population age, and parameter rates
sample_pop = sample_population(n_pop, "data/augmented_population.csv")

transmission_rate = 1.28
progression_rate = 1/5.1
recovery_rate = 0.056 # Approx 1/18 -> Recovery occurs after 18 days
hosp_rate = 1/11.4 #1/6.3 # From Tucker Model
crit_rate = 0.3 # From camp_params

prob_global_contact = 1
prob_detected_global_contact = 1



prob_hosp_to_critical = list(sample_pop["death_rate"]/sample_pop["prob_hospitalisation"])
prob_asymptomatic = list(1 - sample_pop["prob_symptomatic"])
prob_symp_to_hosp = list(sample_pop["prob_hospitalisation"])

init_symp_cases = 1
init_asymp_cases = 0

In [52]:
ref_model = SymptomaticSEIRSNetworkModel(G=graph, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=prob_hosp_to_critical, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True,node_groups=node_groups)

In [53]:
node_states,simulation_results,grouped_data = run_simulation(ref_model, 20, print_info=False, store_every=1)

  0%|          | 0/20 [00:00<?, ?it/s]

Running simulation for 20 steps...



100%|██████████| 20/20 [02:50<00:00,  8.50s/it]


In [56]:
def find_whole_time(tseries, T):
    idt=[]
    for t in range(0, T + 1):
        idt.append(np.abs(tseries - t).argmin())
    return idt

In [51]:
def run_simulation(model, t, print_info=False, store_every=1):
    node_states = dict()
    simulation_results = defaultdict(list)

    print(f"Running simulation for {t} steps...\n")

    for i in tqdm(range(1, t + 1)):
        model.run(T=1, verbose=print_info)

        if i % store_every == 0:
            # Store the node states - an array of size (1, num_nodes) -> we store a copy because the array gets updated
            node_states[i] = np.copy(model.X.T)

            # Store the quantities of the last time step t 
            simulation_results["Symptomatic"].append(model.numS[-1])
            simulation_results["Exposed"].append(model.numE[-1])
            simulation_results["Infected_Presymptomatic"].append(model.numI_pre[-1])
            simulation_results["Infected_Symptomatic"].append(model.numI_S[-1])
            simulation_results["Infected_Asymptomatic"].append(model.numI_A[-1])
            simulation_results["Hospitalized"].append(model.numH[-1])
            simulation_results["Recovered"].append(model.numR[-1])
            simulation_results["Fatalities"].append(model.numF[-1])
            simulation_results["Detected_Presymptomatic"].append(model.numD_pre[-1])
            simulation_results["Detected_Symptomatic"].append(model.numD_S[-1])
            simulation_results["Detected_Asymptomatic"].append(model.numD_A[-1])
            simulation_results["T_index"].append(model.tidx)
    
    group_data=model.nodeGroupData
    time_stamps=model.tseries
    idt=find_whole_time(time_stamps,t)
    timed_group_data=group_data.copy()
    for group,group_detail in group_data.items():
        for key,value in group_detail.items():
            if key!='nodes' and  key!='mask' and key!='N':
                timed_group_data[group][key]=value[idt]
    
    
    return node_states,simulation_results,timed_group_data